# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [9]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [3]:
df = pd.read_csv('data/previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [8]:
df = df[list(df.columns)[3:]]
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  

In [31]:
for i in list(df.columns):
    display(df[i].value_counts().to_frame())

,sexo
F,10119
M,4881


,posse_de_veiculo
False,9140
True,5860


,posse_de_imovel
True,10143
False,4857


,qtd_filhos
0,10376
1,3037
2,1376
3,185
4,17
7,5
14,2
5,2


,tipo_renda
Assalariado,7633
Empresário,3508
Pensionista,2582
Servidor público,1268
Bolsista,9


...

,tipo_residencia
Casa,13532
Com os pais,675
Governamental,452
Aluguel,194
Estúdio,83
Comunitário,64


,idade
40,538
37,469
43,458
32,455
33,441
39,440
38,438
27,436
60,435
46,431


,tempo_emprego
4.216438,38
6.934247,30
7.520548,29
15.449315,27
5.717808,26
...,...
31.202740,1
5.279452,1
0.652055,1
2.827397,1


,qt_pessoas_residencia
2.0,8181
1.0,2752
3.0,2551
4.0,1311
5.0,179
6.0,18
9.0,5
15.0,2
7.0,1


,renda
5402.44,6
1272.04,6
9826.31,6
43433.94,6
728.96,6
...,...
8394.82,1
7827.08,1
5741.98,1
5699.89,1


In [44]:
reg = 'np.log(renda) ~  C(sexo, Treatment(1)) +\
                        C(posse_de_veiculo, Treatment(1)) +\
                        C(posse_de_imovel, Treatment(0)) +\
                        C(qtd_filhos, Treatment(0)) +\
                        C(tipo_renda, Treatment(0)) +\
                        C(educacao, Treatment(2)) +\
                        C(estado_civil, Treatment(0)) +\
                        C(tipo_residencia, Treatment(1)) +\
                        idade +\
                        tempo_emprego +\
                        C(qt_pessoas_residencia, Treatment(2))'


model = smf.ols(reg, df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Sat, 04 Feb 2023   Prob (F-statistic):               0.00
Time:                        09:40:40   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.9245      0.037    211.393      0.000       7.851       7.998
C(sexo, Treatment(1))[T.F]                           -0.7878      0.015    -53.664      0.000      -0.817      -0.759
C(posse_de_veiculo, Treatment(1))[T.False]           -0.0436      0.014     -3.079      0.002      -0.071      -0.016
C(posse_de_imovel, Treatment(0))[T.True]              0.0828      0.014      5.907      0.000       0.055       0.110
C(qtd_filhos, Treatment(0))[T.1]                     -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos, Treatment(0))[T.2]                     -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                     -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos, Treatment(0))[T.4]                      0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos, Treatment(0))[T.5]                      0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos, Treatment(0))[T.7]                      0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos, Treatment(0))[T.14]                    -0.4630      0.261     -1.772      0.076      -0.975       0.049
C(tipo_renda, Treatment(0))[T.Bolsista]               0.2259      0.241      0.937      0.349      -0.247       0.699
C(tipo_renda, Treatment(0))[T.Empresário]             0.1547      0.015     10.359      0.000       0.125       0.184
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.2891      0.243     -1.190      0.234      -0.765       0.187
C(tipo_renda, Treatment(0))[T.Servidor público]       0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment(2))[T.Primário]                 0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment(2))[T.Pós graduação]            0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment(2))[T.Superior completo]        0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.897      0.370      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]             0.3176      0.112      2.831      0.005       0.098       0.537
C(estado_civil, Treatment(0))[T.Solteiro]             0.2556      0.110      2.319      0.020       0.040       0.472
C(estado_civil, Treatment(0))[T.União]               -0.0341      0.025     -1.352      0.176      -0.083       0.015
C(estado_civil, Treatment(0))[T.Viúvo]                0.3535      0.117      3.017      0.003       0.124       0.583
C(tipo_residenc

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  

In [46]:
reg = 'np.log(renda) ~  C(sexo, Treatment(1)) +\
                        C(posse_de_veiculo, Treatment(1)) +\
                        C(posse_de_imovel, Treatment(0)) +\
                        qtd_filhos +\
                        C(tipo_renda, Treatment(0)) +\
                        C(estado_civil, Treatment(0)) +\
                        idade +\
                        tempo_emprego +\
                        qt_pessoas_residencia'


model = smf.ols(reg, df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Sat, 04 Feb 2023   Prob (F-statistic):               0.00
Time:                        09:43:03   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                           7.4305      0.218     34.032      0.000       7.002       7.858
C(sexo, Treatment(1))[T.F]                         -0.7819      0.015    -53.480      0.000      -0.811      -0.753
C(posse_de_veiculo, Treatment(1))[T.False]         -0.0535      0.014     -3.789      0.000      -0.081      -0.026
C(posse_de_imovel, Treatment(0))[T.True]            0.0848      0.014      6.172      0.000       0.058       0.112
C(tipo_renda, Treatment(0))[T.Bolsista]             0.2998      0.242      1.241      0.215      -0.174       0.773
C(tipo_renda, Treatment(0))[T.Empresário]           0.1655      0.015     11.120      0.000       0.136       0.195
C(tipo_renda, Treatment(0))[T.Pensionista]         -0.2540      0.242     -1.051      0.293      -0.728       0.220
C(tipo_renda, Treatment(0))[T.Servidor público]     0.0769      0.022      3.472      0.001       0.033       0.120
C(estado_civil, Treatment(0))[T.Separado]           0.3241      0.112      2.907      0.004       0.106       0.543
C(estado_civil, Treatment(0))[T.Solteiro]           0.2623      0.109      2.403      0.016       0.048       0.476
C(estado_civil, Treatment(0))[T.União]             -0.0361      0.025     -1.431      0.153      -0.086       0.013
C(estado_civil, Treatment(0))[T.Viúvo]              0.3588      0.116      3.097      0.002       0.132       0.586
qtd_filhos                                         -0.2560      0.108     -2.366      0.018      -0.468      -0.044
idade                                               0.0048      0.001      6.147      0.000       0.003       0.006
tempo_emprego                                       0.0617      0.001     59.407      0.000       0.060       0.064
qt_pessoas_residencia                               0.2839      0.108      2.629      0.009       0.072       0.496
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                1.121
Skew:                           0.021   Prob(JB):                        0.571
Kurtosis:                       3.019   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [52]:
df_aux = df[~df['tipo_renda'].isin(['Bolsista', 'Pensionista']) & ~df['estado_civil'].isin(['União'])]


reg = 'np.log(renda) ~  C(sexo, Treatment(1)) +\
                        C(posse_de_veiculo, Treatment(1)) +\
                        C(posse_de_imovel, Treatment(0)) +\
                        qtd_filhos +\
                        C(tipo_renda, Treatment(0)) +\
                        C(estado_civil, Treatment(0)) +\
                        idade +\
                        tempo_emprego +\
                        qt_pessoas_residencia'


model = smf.ols(reg, df_aux).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     532.9
Date:                Sat, 04 Feb 2023   Prob (F-statistic):               0.00
Time:                        09:46:56   Log-Likelihood:                -12581.
No. Observations:               11485   AIC:                         2.519e+04
Df Residuals:                   11472   BIC:                         2.528e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                           7.3949      0.229     32.321      0.000       6.946       7.843
C(sexo, Treatment(1))[T.F]                         -0.7942      0.015    -52.285      0.000      -0.824      -0.764
C(posse_de_veiculo, Treatment(1))[T.False]         -0.0493      0.015     -3.361      0.001      -0.078      -0.021
C(posse_de_imovel, Treatment(0))[T.True]            0.0846      0.014      5.898      0.000       0.056       0.113
C(tipo_renda, Treatment(0))[T.Empresário]           0.1744      0.016     11.219      0.000       0.144       0.205
C(tipo_renda, Treatment(0))[T.Servidor público]     0.0795      0.023      3.471      0.001       0.035       0.124
C(estado_civil, Treatment(0))[T.Separado]           0.3456      0.117      2.964      0.003       0.117       0.574
C(estado_civil, Treatment(0))[T.Solteiro]           0.2823      0.114      2.470      0.014       0.058       0.506
C(estado_civil, Treatment(0))[T.Viúvo]              0.3833      0.121      3.174      0.002       0.147       0.620
qtd_filhos                                         -0.2782      0.113     -2.452      0.014      -0.500      -0.056
idade                                               0.0047      0.001      5.859      0.000       0.003       0.006
tempo_emprego                                       0.0619      0.001     57.632      0.000       0.060       0.064
qt_pessoas_residencia                               0.3043      0.113      2.687      0.007       0.082       0.526
==============================================================================
Omnibus:                        1.064   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.587   Jarque-Bera (JB):                1.043
Skew:                           0.022   Prob(JB):                        0.594
Kurtosis:                       3.013   Cond. No.                     2.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""